In [1]:
!pip install errant

     |████████████████████████████████| 501kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 1.5MB 12.1MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144795 sha256=8a220d6a01e4f577f972398b1e4f8d1d59d0851004dbec63ad93da0672afa3e4
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=dffd74e748c9feb6375d2a18c4d68eb4853f66a352781e94092d7283c0c1bc30
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built python-Levenshtein nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
import errant
import pandas as pd
import difflib

In [3]:
def get_diff(a,b):
    if len(a)>len(b): 
        res=''.join(a.split(b))             
    else: 
        res=''.join(b.split(a))             
    return res.strip()

In [4]:
punctuation = ['.',',','!','?','..','...','-']

In [6]:
def error_type(s1,s2):
    annotator = errant.load('en')
    orig = annotator.parse(s1)
    cor = annotator.parse(s2)
    edits = annotator.annotate(orig, cor)
    for e in edits:
        if 'DET' in e.type:
            return 'Articles'
        elif 'PREP' in e.type or 'PART' in e.type:
            return 'Preposition'
        elif 'PUNCT' in e.type or (e.o_str!='' and e.c_str!='' and get_diff(e.o_str,e.c_str) in punctuation) or ('-' in e.o_str or '-' in e.c_str):
            return 'Punctuation'
        elif 'VERB' in e.type:
            if 'SVA' in e.type:
                return 'Subject Verb Agreement'
            else:
                return 'Verb Form'
        elif 'NOUN' in e.type or 'ADJ' in e.type or 'MORPH' in e.type:
            return 'Word Form'
        else:
            return 'Other'

In [7]:
def get_action(s1,s2):
    annotator = errant.load('en')
    orig = annotator.parse(s1)
    cor = annotator.parse(s2)
    edits = annotator.annotate(orig, cor)
    for e in edits:
        if 'R:' in e.type:
            return 'replace'
        elif 'M:' in e.type:
            return 'insert'
        elif 'U:' in e.type:
            return 'remove'

In [10]:
data = pd.read_csv('ErrorData.csv')

In [9]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ErrorData.csv to ErrorData.csv
User uploaded file "ErrorData.csv" with length 101264 bytes


In [31]:
data.head(30)

,questionid,category,round,question,solution,explanation,type,action,give_action,give_category
0,1,Articles,1,These devices generate huge amount of data tha...,These devices generate a huge amount of data t...,Article required,C,insert,insert,Articles
1,1,Articles,1,Fashion items make up large portion of e-comme...,Fashion items make up a large portion of e-com...,"The word ""portion"" is a single, countable noun...",SP,insert,insert,Articles
2,1,Articles,2,"Firstly, increasing number of people are reviv...","Firstly, an increasing number of people are re...",Article required,C,insert,insert,Articles
3,1,Articles,3,"In the recent years, the rise of technology in...","In recent years, the rise of technology in the...",No article required,C,remove,remove,Articles
4,1,Articles,4,"The Sugar Detox Diet, which aims to promote a ...","The Sugar Detox Diet, which aims to promote he...",No article required,C,remove,remove,Articles
5,1,Preposition,1,A study by ABI Research (2017) forecasted that...,A study by ABI Research (2017) forecasted that...,Idiomatic expressions: will grow by and will g...,C,insert,insert,Preposition
6,1,Preposition,1,One method to measure brand positioning would ...,One method to measure brand positioning would ...,"Modals or modal auxiliary verbs (can, could, m...",SP,insert,insert,Verb Form
7,1,Preposition,2,I have researched on recent health food trends.,I have researched recent health food trends.,No preposition required after research as a verb,C,remove,remove,Preposition
8,1,Preposition,3,Board games help people to build up social ski...,Board games help people to build social skills...,No preposition required because the direction ...,C,remove,remove,Preposition
9,1,Preposition,4,The cost of a mobile phone can amount from $1000.,The cost of a mobile phone can amount to $1000.,"Amount to means up to an amount, as in this case.",C,replace,replace,Preposition


In [12]:
data['give_action'] = data.apply(lambda x: get_action(x.question, x.solution),axis=1)

In [13]:
data['give_category'] = data.apply(lambda x: error_type(x.question, x.solution),axis=1)

In [18]:
s1 = data['question'].iloc[0]
s1

"These devices generate huge amount of data that could be used by doctors to enhance patients' care (Burrus, 2015)."

In [19]:
s2 = data['solution'].iloc[0]
s2

"These devices generate a huge amount of data that could be used by doctors to enhance patients' care (Burrus, 2015)."

In [81]:
def get_expla(s1,s2):
    annotator = errant.load('en')
    orig = annotator.parse(s1)
    cor = annotator.parse(s2)
    edits = annotator.annotate(orig, cor)
    error = error_type(s1,s2)
    if error =='Articles':
        for e in edits:
            if 'R:' in e.type:
                expla = f"Consider article '{e.c_str}' in front of countable or singular nouns referring to people or things what have not already been mentioned."
                return expla
            elif 'M:' in e.type:
                expla = f"Article '{e.c_str}' is required because of the countable or singular nouns referring to people or things what have not already been mentioned."
                return expla
            elif 'U:' in e.type:
                return 'No article required'


    elif error =='Preposition':
        for e in edits:
            if 'R:' in e.type:
                expla = f"Consider '{e.c_str}' to be the proper preposition."
                return expla
            elif 'M:' in e.type:
                expla = f"You need a preposition '{e.c_str}'before a noun or pronoun to show place, position, time or method."
                return expla
            elif 'U:' in e.type:
                expla = f"You don't need preposition '{e.o_str}' here, consider to remove it."
                return expla
            else:
                return 'Others'

In [82]:
get_expla(s1,s2)

"Article 'a' is required because of the countable or singular nouns referring to people or things what have not already been mentioned."

In [83]:
s3 = data['question'].iloc[8]
s4 = data['solution'].iloc[8]

In [84]:
error_type(s3,s4)

'Preposition'

In [85]:
get_expla(s3,s4)

"You don't need preposition 'up' here, consider to remove it."